In [51]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [26]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root = '../dataset/mnist/',
                              train=True,
                              download = True,
                              transform = transform)
train_loader = DataLoader(train_dataset,
                         shuffle = True,
                         batch_size = batch_size)
test_dataset = datasets.MNIST(root = '../dataset/mnist/',
                             train = False,
                             download = True,
                             transform = transform)
test_loader = DataLoader(test_dataset,
                        shuffle = False,
                        batch_size = batch_size)

In [27]:
class InceptionA(torch.nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch_pool = torch.nn.Conv2d(in_channels, 24, kernel_size = 1)
        
        self.branch1x1 = torch.nn.Conv2d(in_channels, 16, kernel_size = 1)
        
        self.branch5x5_1 = torch.nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.branch5x5_2 = torch.nn.Conv2d(16, 24, kernel_size = 5, padding = 2)
        
        self.branch3x3_1 = torch.nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.branch3x3_2 = torch.nn.Conv2d(16, 24, kernel_size = 3, padding = 1)
        self.branch3x3_3 = torch.nn.Conv2d(24, 24, kernel_size = 3, padding = 1)
    def forward(self, x):
        branch_pool = F.avg_pool2d(x, kernel_size = 3, stride = 1, padding = 1)
        branch_pool = self.branch_pool(branch_pool)
        
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, dim = 1)


In [28]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size = 5)
        self.incep1 = InceptionA(in_channels = 10)
        self.incep2 = InceptionA(in_channels = 20)
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(1408, 10)
        
    def forward(self,x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

In [33]:
model = Net()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(88, 20, kernel_size=(5, 5), stride=(1, 1))
  (incep1): InceptionA(
    (branch_pool): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch1x1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (incep2): InceptionA(
    (branch_pool): Conv2d(20, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch1x1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   

In [34]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [35]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        
        #forward
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch +1, batch_idx +1, running_loss / 300))
            running_loss = 0.0

In [36]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim = 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
        print('Accuracy on test set: %d %%' % (100 * correct / total))
    return 100 * correct / total

In [37]:
if __name__ == '__main__':
    correct = []
    for epoch in range(10):
        train(epoch)
        correct.append(test())


[1,   300] loss: 1.153
[1,   600] loss: 0.221
[1,   900] loss: 0.150
Accuracy on test set: 96 %
[2,   300] loss: 0.112
[2,   600] loss: 0.106
[2,   900] loss: 0.096
Accuracy on test set: 97 %
[3,   300] loss: 0.081
[3,   600] loss: 0.081
[3,   900] loss: 0.072
Accuracy on test set: 97 %
[4,   300] loss: 0.063
[4,   600] loss: 0.065
[4,   900] loss: 0.063
Accuracy on test set: 98 %
[5,   300] loss: 0.058
[5,   600] loss: 0.053
[5,   900] loss: 0.054
Accuracy on test set: 98 %
[6,   300] loss: 0.047
[6,   600] loss: 0.050
[6,   900] loss: 0.050
Accuracy on test set: 98 %
[7,   300] loss: 0.041
[7,   600] loss: 0.044
[7,   900] loss: 0.045
Accuracy on test set: 98 %
[8,   300] loss: 0.040
[8,   600] loss: 0.039
[8,   900] loss: 0.042
Accuracy on test set: 98 %
[9,   300] loss: 0.036
[9,   600] loss: 0.037
[9,   900] loss: 0.039
Accuracy on test set: 98 %
[10,   300] loss: 0.034
[10,   600] loss: 0.035
[10,   900] loss: 0.033
Accuracy on test set: 98 %


NameError: name 'plot' is not defined

In [1]:
x = np.linspace(0,9,10)
correct = np.array(correct)
plt.plot(x, correct, c = 'b')
plt.figure(figsize=(10,6)
plt.title("google net")
plt.xlabel("epoch")
plt.ylabel("correct")

SyntaxError: invalid syntax (<ipython-input-1-fff83adb1edf>, line 5)

In [54]:
print(correct)

[96.6  97.69 97.55 98.3  98.43 98.05 98.72 98.53 98.63 98.5 ]
